In [51]:
# Set up dependencies and import necessary functions

import sys
import os
import requests
from pypdf import PdfReader
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install sentence-transformers pypdf

In [52]:
API_URL = "http://127.0.0.1:8080/v1/chat/completions" # This is the URL of your local llama.cpp server which connects to the api

def ask_nor_model(prompt, system="You are a helpful assistant.", max_tokens=200):
    payload = {                                   # <---- this builds the json data which will be send to the NorMistral API
        "model": "normistral-7b-warm-instruct",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens
    }

    res = requests.post(API_URL, json=payload)  #sends HTTP post request to the running LLM server
    if not res.ok:
        # Print server message to see what went wrong
        print("Server response:", res.status_code, res.text)
        res.raise_for_status()
   
   
    return res.json()["choices"][0]["message"]["content"].strip() # Convert the server's JSON reply back to Python object


In [53]:
# Folder path of research papers is set and for each PDF, reads each line, extract text and store all pages together in a single big string
PAPERS_DIR = r"C:\llama_cpp\papers"  

def load_pdfs_from_folder(folder):
    docs = []
    for fname in os.listdir(folder):
        if fname.lower().endswith(".pdf"):
            path = os.path.join(folder, fname)
            reader = PdfReader(path)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text() or ""
                text += page_text + "\n"
            docs.append((fname, text))
    return docs



In [54]:
# Split Long text into smaller overlapping chunks

def chunk_text(text, chunk_size=400, overlap=100):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

docs = load_pdfs_from_folder(PAPERS_DIR)

all_chunks = []
chunk_sources = []  # to remember which PDF each chunk came from


#For each document: Split its text into chunks, add each chunk to the master list and record which file it belongs to.

for fname, text in docs:
    chunks = chunk_text(text)
    for c in chunks:
        all_chunks.append(c)
        chunk_sources.append(fname)

print(f"Loaded {len(docs)} PDF(s), total {len(all_chunks)} chunks.") # tells how many PDFs and chunks were created

Loaded 3 PDF(s), total 327 chunks.


In [55]:
# Finds the most relevant text chunks from your papers for a given question

def retrieve_chunks(query, top_k=5):
    # encode query
    q_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    # cosine similarity via dot product (normalized vectors)
    sims = np.dot(chunk_embeddings, q_emb)
    # get best indices
    top_idx = np.argsort(-sims)[:top_k]
    
    results = []
    for i in top_idx:
        results.append({
            "source": chunk_sources[i],
            "score": float(sims[i]),
            "text": all_chunks[i]
        })
    return results

In [56]:
# This method ensures that the model’s answers are based only on your PDFs instead of its general training knowledge.
# So now, instead of a “chatbot guessing,” we have a mini research assistant that reads your uploaded papers and answers precisely from them.

def rag_ask(question, top_k=3, max_tokens=200):
    retrieved = retrieve_chunks(question, top_k=top_k)
    context_blocks = []
    for r in retrieved:
        context_blocks.append(f"[Source: {r['source']}]\n{r['text']}")
    context = "\n\n---\n\n".join(context_blocks)

    prompt = (
        "You are given excerpts from research papers.\n"
        "Answer the question ONLY using this context. "
        "If the answer is not clearly in the context, say you are not sure.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n\n"
        "Answer in a clear, concise way:"
    )
    
    # sending the prompt to the LLM for generation part of the RAG
    return ask_nor_model(
        prompt,
        system="You are a careful research assistant.",
        max_tokens=max_tokens,
    )


In [57]:
response = ask_nor_model("Hva er kunstig intelligens?")
print(response)


Kunstig intelligens (AI) er et tverrfaglig felt som kombinerer datavitenskap, matematikk, psykologi og andre disipliner for å skape intelligente systemer som kan etterligne menneskelig intelligens. Disse systemene kan lære av erfaring, tilpasse seg nye situasjoner og ta beslutninger basert på analyse av data.

AI brukes i en rekke applikasjoner, inkludert:
1. Bilde- og talegjenkjenning: AI kan analysere og tolke bilder og tale, gjenkjenne objekter, ansikter og stemmer, og trekke ut meningsfull informasjon.
2. Naturlig språkbehandling: AI kan forstå, tolke og generere menneskelig språk, noe som muliggjør taleassistenter, virtuelle assistenter og andre applikasjoner.
3. Robotikk: AI brukes i roboter, droner og andre autonome systemer for å la dem lære av miljøet og ta beslutninger basert på deres oppfatning av omgivelsene.
4. Maskinlæring: AI bruker statistiske algoritmer for å analysere data, identifisere mønstre og gjøre spådommer eller ta avgjørelser basert på den data


In [64]:
response = ask_nor_model("Can you list some LLMs for Physiotherapy that you know?")
print(response)


Here are some LLMs that I have encountered in my research:

1. The University of Texas at Austin: The University of Texas at Austin has a Physiotherapy LLM program that focuses on clinical practice and research.
2. The University of Michigan: The University of Michigan's School of Public Health has a Physiotherapy LLM program that provides students with a strong foundation in clinical skills, health education, and research.
3. The University of Rochester: The University of Rochester has a Physiotherapy LLM program that emphasizes clinical practice, research, and health policy.
4. The University of California, Berkeley: The University of California, Berkeley has a Physiotherapy LLM program that offers a comprehensive curricul


In [65]:
answer = rag_ask("Can you list some LLMs for Physiotherapy that you know?")
print(answer)

Here are some LLMs that have been developed for physiotherapy, according to the provided context:

1. ChatGPT: This is a language model developed by OpenAI that can generate responses to clinical vignettes. It has been used in medical schools to support case-based learning, diagnostic reasoning, and simulating patient interactions.
2. DL4PHYSIO: This is a physiotherapy-specific LLM developed on top of the OpenAI GPT-3 model. It has been trained on a specialized training dataset to provide tailored recommendations for managing conditions like low back pain or knee osteoarthritis.
3. MarGPT: This is another language model developed by OpenAI that can generate responses to clinical vignettes. It
